# Capstone project:
# La Jolla Rental House Prices in relation to Venues
### Author: James Lee

## 1. Background

San Diego is a city on Pacific coast of California with around 1.5  million residents. Because of its warm climate and beaches, San Diego has one of the highest cost of living in California. La Jolla is a seaside neighborhood where UC San Diego is located at and has an even higher cost of living compared to other parts of San Diego. <br> 

The main purpose of this project is to analyze the relationships between the venues and rental prices across different areas near UCSD.
<br> 


## 2. Dataset

* I will be obtaining all the data for venues in San Diego through **Foursquare API**.
<br> <br> 
* Data for neighborhoods and zip codes are collectedfrom SANDAG. <br> 
Link Address: https://opendata.arcgis.com/datasets/41c3a7bd375547069a78fce90153cbc0_5.csv?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D

<br> <br> 
* The rental prices dataset was obtained from Zillow. The particular dataset does not include raw rental prices. The values are weighted to rental housing stock to ensure representativeness acorss the entire market. The values are caculated so that they fall into the 40-60th percentile rnage for all rents in a given region in order to represent the average rent prices.
<br>
Link Address: http://files.zillowstatic.com/research/public_v2/zori/Metro_ZORI_AllHomesPlusMultifamily_SSA.csv  

<br>


1) Import all the necessary libraries

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

print('Libraries imported.')

Libraries imported.


First, get all the region info by obatin zip codes and communities

In [141]:
district = pd.read_csv('https://opendata.arcgis.com/datasets/41c3a7bd375547069a78fce90153cbc0_5.csv?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D')
district = district[['ZIP', 'COMMUNITY']]
district.head()

,ZIP,COMMUNITY
0,91901,Alpine
1,91902,Bonita
2,91905,Boulevard
3,91906,Campo
4,91910,Chula Vista


Now import all the adjusted rent data from Zillow along with zip codes

In [94]:
rent_temp = pd.read_csv('http://files.zillowstatic.com/research/public_v2/zori/Zip_ZORI_AllHomesPlusMultifamily_SSA.csv')
rent_temp = rent_temp[rent_temp.MsaName == 'San Diego, CA']
rent_temp.shape

(68, 86)

In [149]:
rent = rent_temp.iloc[:, [1, 85]] 
rent.columns = ['ZIP', 'rent']
rent.shape

(68, 2)

Import latitude and longitudes info and match with rent dataframe

In [116]:
coordinates = pd.read_csv('https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/download/?format=csv&timezone=America/Los_Angeles&lang=en&use_labels_for_header=true&csv_separator=%3B', sep=';')
coordinates = coordinates[['Zip', 'Latitude', 'Longitude']]
coordinates.columns = ['ZIP', 'lat', 'lng']

In [117]:
coordinates.head()

,ZIP,lat,lng
0,66025,38.917032,-95.06455
1,74565,34.831398,-95.83967
2,75631,32.237924,-94.46427
3,92067,33.016492,-117.20264
4,92119,32.802250,-117.02431


In [180]:
merged_df1 = rent.merge(coordinates, how='inner', on='ZIP')
merged_df1.head()

,ZIP,rent,lat,lng
0,91910,2117.0,32.638654,-117.06686
1,92126,2463.0,32.914346,-117.14116
2,91911,2037.0,32.608799,-117.06070
3,92101,2340.0,32.719601,-117.16246
4,92109,2322.0,32.791246,-117.24343


In [181]:
merged_df1.shape

(65, 4)

In [227]:
merged_df = merged_df1.merge(district, how='inner', on='ZIP')
merged_df.head()

,ZIP,rent,lat,lng,COMMUNITY
0,91910,2117.0,32.638654,-117.06686,Chula Vista
1,92126,2463.0,32.914346,-117.14116,San Diego
2,91911,2037.0,32.608799,-117.06070,Chula Vista
3,92101,2340.0,32.719601,-117.16246,San Diego
4,92109,2322.0,32.791246,-117.24343,San Diego


2) Input the Foursquare API info below

In [183]:
CLIENT_ID = 'O5XRHQWTDAQI4X53O3LXOAQOUYSIJ5UTGL1W21E43AT3PDCT' # your Foursquare ID
CLIENT_SECRET = '5U5LXOZT5B4FJNIMTQCPX5I4GTR4WNAIT2APEDB2NJG3TYV2' # your Foursquare Secret
ACCESS_TOKEN = '2VMIKNFIAIA3E4RFB1EJHDPNRHDJK1VRT3D1IKVULUZV20ZY' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O5XRHQWTDAQI4X53O3LXOAQOUYSIJ5UTGL1W21E43AT3PDCT
CLIENT_SECRET:5U5LXOZT5B4FJNIMTQCPX5I4GTR4WNAIT2APEDB2NJG3TYV2


Get the center latitude and longitude of UCSD

In [210]:
address = 'UCSD'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

32.87935255 -117.2311004938553


Use folium library to generate a map of La Jolla 

In [185]:
# create map using latitude and longitude values
map_LaJolla = folium.Map(location=[latitude, longitude], zoom_start=10)

# add a red circle marker to represent UCSD
folium.CircleMarker(
    [latitude, longitude],
    radius=15,
    color='red',    
    popup='UCSD',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_LaJolla)

# add markers to map
for lat, lng, ZIP, COMMUNITY in zip(merged_df['lat'], merged_df['lng'], merged_df['ZIP'], merged_df['COMMUNITY']):
    label = '{}, {}'.format(ZIP, COMMUNITY)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill = True,
        fill_color = 'blue',
        fill_opacity = 0.6,
        parse_html=False).add_to(map_LaJolla)  

map_LaJolla

In [245]:
merged_df.head()

,ZIP,rent,lat,lng,COMMUNITY
0,91910,2117.0,32.638654,-117.06686,Chula Vista
1,92126,2463.0,32.914346,-117.14116,San Diego
2,91911,2037.0,32.608799,-117.06070,Chula Vista
3,92101,2340.0,32.719601,-117.16246,San Diego
4,92109,2322.0,32.791246,-117.24343,San Diego


In [289]:
# convert ZIP's data type to 'object' first
final_df = merged_df
final_df['ZIP'] = final_df['ZIP'].astype('object')
final_df.head()

,ZIP,rent,lat,lng,COMMUNITY
0,91910,2117.0,32.638654,-117.06686,Chula Vista
1,92126,2463.0,32.914346,-117.14116,San Diego
2,91911,2037.0,32.608799,-117.06070,Chula Vista
3,92101,2340.0,32.719601,-117.16246,San Diego
4,92109,2322.0,32.791246,-117.24343,San Diego


3) Now let's search up to 50 venues in each region (500m radius from UCSD)  <br>
(The only issue with free Foursquare API is that the limit of query result is set to 50...)

In [292]:
search_query = ''
radius = 500
LIMIT = 50
venue_df = pd.DataFrame([])

# create a new dataframe with venues in each region
for lat, lng in zip(final_df['lat'], final_df['lng']):
    latitude = lat
    longitude = lng
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude,           longitude, ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json() 
    venues = results['response']['venues']
    temp_df = json_normalize(venues)
    temp_df = temp_df[['name', 'categories', 'location.postalCode']]
    temp_df.columns = ['venue', 'categories', 'ZIP']
    venue_df = venue_df.append(temp_df)
    


KeyError: 'venues'

In [287]:
venue_df.head()

""


In [193]:
search_query = ''
radius = 100

In [194]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()

In [285]:
# assign relevant part of JSON to venues
venues = results['response']['venues']
# tranform venues into a dataframe
df = json_normalize(venues)
#df['location.postalCode']
df.categories

0     [{'id': '4bf58dd8d48988d1e5941735', 'name': 'D...
1     [{'id': '4bf58dd8d48988d162941735', 'name': 'O...
2     [{'id': '4f4533814b9074f6e4fb0106', 'name': 'M...
3     [{'id': '4bf58dd8d48988d163941735', 'name': 'P...
4     [{'id': '4bf58dd8d48988d132941735', 'name': 'C...
5     [{'id': '52e81612bcbc57f1066b7a35', 'name': 'C...
6     [{'id': '4bf58dd8d48988d124941735', 'name': 'O...
7     [{'id': '4f2a210c4b9023bd5841ed28', 'name': 'H...
8     [{'id': '4bf58dd8d48988d17c941735', 'name': 'C...
9     [{'id': '4bf58dd8d48988d132941735', 'name': 'C...
10                                                   []
11    [{'id': '4bf58dd8d48988d132941735', 'name': 'C...
12    [{'id': '5744ccdfe4b0c0459246b4ac', 'name': 'K...
13    [{'id': '4eb1bea83b7b6f98df247e06', 'name': 'F...
14    [{'id': '4bf58dd8d48988d104941735', 'name': 'M...
15    [{'id': '4bf58dd8d48988d178941735', 'name': 'D...
16    [{'id': '4bf58dd8d48988d176941735', 'name': 'G...
17    [{'id': '4bf58dd8d48988d1e8941735', 'name'

In [171]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
df_update = df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_update['categories'] = df_update.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_update.columns = [column.split('.')[-1] for column in df_update.columns]



KeyError: "None of [Index(['name', 'categories', 'id'], dtype='object')] are in the [columns]"

In [52]:
df_update.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,BJ's Restaurant & Brewhouse,Brewery,8785 Villa La Jolla Dr,at Nobel Dr,32.868009,-117.233232,"[{'label': 'display', 'lat': 32.86800867565035...",1278,92037,US,San Diego,CA,United States,"[8785 Villa La Jolla Dr (at Nobel Dr), San Die...",58cdfbba110d037d32772e8c
1,Leucadia Pizzeria & Italian Restaurant,Italian Restaurant,7748 Regents Rd Ste 303,at Arriba St,32.862367,-117.224743,"[{'label': 'display', 'lat': 32.86236681380613...",1982,92122,US,San Diego,CA,United States,"[7748 Regents Rd Ste 303 (at Arriba St), San D...",43f66fd0f964a520b32f1fe3
2,Shores Restaurant,New American Restaurant,8110 Camino Del Oro,NaN,32.855336,-117.258289,"[{'label': 'display', 'lat': 32.85533644046734...",3689,92037,US,La Jolla,CA,United States,"[8110 Camino Del Oro, La Jolla, CA 92037]",4c424971d7fad13ad4f308da
3,Indulge Restaurant & Lounge,New American Restaurant,4550 La Jolla Village Dr,NaN,32.874462,-117.208366,"[{'label': 'display', 'lat': 32.8744618808593,...",2194,92122,US,San Diego,CA,United States,"[4550 La Jolla Village Dr, San Diego, CA 92122]",4b22b29cf964a520074c24e3
4,Barcino Grill Restaurant In Hyatt Regency Aven...,American Restaurant,3777 La Jolla Village Dr,btw I-5 and Lebon,32.870930,-117.225639,"[{'label': 'display', 'lat': 32.87092971801758...",1067,92037,US,La Jolla,CA,United States,"[3777 La Jolla Village Dr (btw I-5 and Lebon),...",4c55a45006901b8d5474d44d


Now let's visualize all the restaurants that are nearby

In [36]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent UCSD
folium.CircleMarker(
    [latitude, longitude],
    radius=15,
    color='red',    
    popup='UCSD',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the restaurants as blue circle markers
for lat, lng, label in zip(df_update.lat, df_update.lng, df_update.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,        
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

(123, 2)

(68, 86)

,ZIP,rent
115,91910,2117.0
162,92126,2463.0
168,91911,2037.0
174,92101,2340.0
205,92109,2322.0
